# Execution num1:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sheila/

In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 50,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None


Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:44916
Please start proper ns-3 simulation script using ./waf --run "..."
40.790771|0|5|-nan
Observation space shape: (1, 300)
Action space shape: (1, 1)


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, state_size=50, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
'''logger = teacher.eval(agent,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        tags=tags,
                        parameters=hyperparams)
'''
# agent.save()


# %%




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/374fca6438734d1e91867167024cb4e8



0
40.790771|0|5|-nan


...

COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/374fca6438734d1e91867167024cb4e8
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2601.197265625, 3158.89453125)
COMET INFO:     Current throughput [89985]  : (17.8710994720459, 59.466060638427734)
COMET INFO:     Fairness index [89985]      : (0.025640999898314476, 0.9967309832572937)
COMET INFO:     Megabytes sent [89985]      : (0.17871099710464478, 2266.769472658634)
COMET INFO:     Observation 0 [89985]       : (0.04930915126266579, 0.9673228760560353)
COMET INFO:     Observation 1 [89985]       : (0.003994262105884034, 0.017204633795323894)
COMET INFO:     Per-ep reward [89985]       : (91.4404296875, 3158.89453125)
COMET INFO:     Round megabytes sent [89985]: (0.0, 15.87176513671875)
COMET INFO:

Sent 2264.69 Mb/s.	Mean speed: 37.74 Mb/s	Episode 15/15 finished



COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


Training finished.


'logger = teacher.eval(agent,\n                        simTime=simTime,\n                        stepTime=stepTime,\n                        history_length=history_length,\n                        tags=tags,\n                        parameters=hyperparams)\n'

# Execution num2:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sheila/

In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 50,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:34185
Please start proper ns-3 simulation script using ./waf --run "..."
40.790771|0|5|-nan
Observation space shape: (1, 300)
Action space shape: (1, 1)


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
#logger = teacher.eval(agent,
#                        simTime=simTime,
#                        stepTime=stepTime,
#                        history_length=history_length,
#                        tags=tags,
 #                       parameters=hyperparams)
# agent.save()


# %%




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/4f407e0e29ac47fe9ca20ca1334450ae



0
40.790771|0|5|-nan


...

Waiting for simulation script to connect on port: tcp://localhost:37819
Please start proper ns-3 simulation script using ./waf --run "..."


COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/4f407e0e29ac47fe9ca20ca1334450ae
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2572.69287109375, 3157.13720703125)
COMET INFO:     Current throughput [89985]  : (15.637200355529785, 51.088985443115234)
COMET INFO:     Fairness index [89985]      : (0.025543000549077988, 0.9963189959526062)
COMET INFO:     Megabytes sent [89985]      : (0.15637199580669403, 2263.373951345682)
COMET INFO:     Observation 0 [89985]       : (0.04867156986147165, 0.9672454253832499)
COMET INFO:     Observation 1 [89985]       : (0.003994262105884034, 0.018906017741214132)
COMET INFO:     Per-ep reward [89985]       : (87.53857421875, 3157.13720703125)
COMET INFO:     Round megabytes sent [89985]: (0.0, 11.839599609375)
CO

Sent 2257.70 Mb/s.	Mean speed: 37.63 Mb/s	Episode 15/15 finished



COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


Training finished.


# Execution num3:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sheila/

In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 50,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:44958
Please start proper ns-3 simulation script using ./waf --run "..."
40.790771|0|5|-nan
Observation space shape: (1, 300)
Action space shape: (1, 1)


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
#logger = teacher.eval(agent,
#                        simTime=simTime,
#                        stepTime=stepTime,
#                        history_length=history_length,
#                        tags=tags,
#                        parameters=hyperparams)
# agent.save()


# %%




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/96c6d414b85c4c0e828330cadc27b5b5



0
40.790771|0|5|-nan


...

Waiting for simulation script to connect on port: tcp://localhost:18750
Please start proper ns-3 simulation script using ./waf --run "..."
Sent 2260.67 Mb/s.	Mean speed: 37.68 Mb/s	Episode 14/15 finished

14
Turning off noise
40.790771|0|5|-nan


 81%|████████▏ | 5126/6300 [50:48<13:20,  1.47it/s, curr_speed=37.08 Mbps, mb_sent=1826.12 Mb]COMET ERROR: File could not be uploaded
COMET ERROR: File could not be uploaded
100%|██████████| 6300/6300 [1:03:35<00:00,  1.65it/s, curr_speed=37.59 Mbps, mb_sent=2265.80 Mb]


Waiting for simulation script to connect on port: tcp://localhost:31631
Please start proper ns-3 simulation script using ./waf --run "..."


COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/96c6d414b85c4c0e828330cadc27b5b5
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2610.87744140625, 3160.368896484375)
COMET INFO:     Current throughput [89985]  : (24.945077896118164, 49.32421875)
COMET INFO:     Fairness index [89985]      : (0.025648999959230423, 0.9954389929771423)
COMET INFO:     Megabytes sent [89985]      : (0.3462519943714142, 2265.7977083027363)
COMET INFO:     Observation 0 [89985]       : (0.04462875900169213, 0.9671473860740663)
COMET INFO:     Observation 1 [89985]       : (0.003994262105884034, 0.019740271235216905)
COMET INFO:     Per-ep reward [89985]       : (86.421630859375, 3160.368896484375)
COMET INFO:     Round megabytes sent [89985]: (0.0, 11.80609130859375)
COME

Sent 2265.80 Mb/s.	Mean speed: 37.76 Mb/s	Episode 15/15 finished

Training finished.


# Execution num4:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sheila/

In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 50,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:50358
Please start proper ns-3 simulation script using ./waf --run "..."
40.790771|0|5|-nan
Observation space shape: (1, 300)
Action space shape: (1, 1)


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
#logger = teacher.eval(agent,
#                        simTime=simTime,
#                        stepTime=stepTime,
#                        history_length=history_length,
#                        tags=tags,
#                        parameters=hyperparams)
# agent.save()


# %%




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/78987b3832974b3398fe55f3e38944ea



0
40.790771|0|5|-nan


...

Waiting for simulation script to connect on port: tcp://localhost:52490
Please start proper ns-3 simulation script using ./waf --run "..."


COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/78987b3832974b3398fe55f3e38944ea
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2603.177978515625, 3163.317626953125)
COMET INFO:     Current throughput [89985]  : (23.17657470703125, 59.07512664794922)
COMET INFO:     Fairness index [89985]      : (0.025675000622868538, 0.9942060112953186)
COMET INFO:     Megabytes sent [89985]      : (0.3239139914512634, 2263.4744570851326)
COMET INFO:     Observation 0 [89985]       : (0.044782353773092226, 0.9673490197459856)
COMET INFO:     Observation 1 [89985]       : (0.003994262105884034, 0.019520594415816838)
COMET INFO:     Per-ep reward [89985]       : (96.757080078125, 3163.317626953125)
COMET INFO:     Round megabytes sent [89985]: (0.0, 11.8060913085937

Sent 2262.46 Mb/s.	Mean speed: 37.71 Mb/s	Episode 15/15 finished



COMET INFO: Waiting for completion of the file uploads (may take several seconds)
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uplo

Training finished.


In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sheila/

In [2]:
#%%
scenario = "convergence"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "convergence",
    "nWifi": 50,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:28028
Please start proper ns-3 simulation script using ./waf --run "..."
40.790771|0|5|-nan
Observation space shape: (1, 300)
Action space shape: (1, 1)


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
#logger = teacher.eval(agent,
#                        simTime=simTime,
#                        stepTime=stepTime,
#                        history_length=history_length,
#                        tags=tags,
#                        parameters=hyperparams)
# agent.save()


# %%




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/0030ef7a36e047e3a304e8f9ee7af9f6



0
40.790771|0|5|-nan


...

Waiting for simulation script to connect on port: tcp://localhost:25268
Please start proper ns-3 simulation script using ./waf --run "..."
Sent 2263.23 Mb/s.	Mean speed: 37.72 Mb/s	Episode 14/15 finished

14
Turning off noise
40.790771|0|5|-nan


100%|██████████| 6300/6300 [7:36:46<00:00,  4.35s/it, curr_speed=37.62 Mbps, mb_sent=2262.44 Mb]


Waiting for simulation script to connect on port: tcp://localhost:49673
Please start proper ns-3 simulation script using ./waf --run "..."


COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/0030ef7a36e047e3a304e8f9ee7af9f6
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2621.3916015625, 3156.943603515625)
COMET INFO:     Current throughput [89985]  : (7.818600177764893, 52.697391510009766)
COMET INFO:     Fairness index [89985]      : (0.025669999420642853, 0.9962319731712341)
COMET INFO:     Megabytes sent [89985]      : (0.07818599790334702, 2263.228790447116)
COMET INFO:     Observation 0 [89985]       : (0.044856209950521586, 0.96708333303531)
COMET INFO:     Observation 1 [89985]       : (0.003994262105884034, 0.01850682074250814)
COMET INFO:     Per-ep reward [89985]       : (84.515380859375, 3156.943603515625)
COMET INFO:     Round megabytes sent [89985]: (0.01116899959743023, 11.8

Sent 2262.44 Mb/s.	Mean speed: 37.71 Mb/s	Episode 15/15 finished

Training finished.
